In [1]:
from random import Random
from Crypto.Util.number import bytes_to_long, long_to_bytes
import math
from scipy.special import gammainc

r = Random()

len_bts = 10000
bts = r.randbytes(len_bts)

def long_to_list(n: int, length: int):
    res = []
    for i in range(n.bit_length()):
        res.append( n & 1 )
        n >>= 1
    while len(res) < length:
        res.append(0)

    return res

In [2]:
# Frequency test

# returns true for random sequences
def frequency_test(b: list, n: int) -> bool:
    s = sum([1 if b[i] == 1 else -1 for i in range(n)])

    sobs = abs(s) / (n ** 0.5)

    p = math.erfc(sobs / (2 ** 0.5))

    return p >= 0.01

eps = int("1100100100001111110110101010001000100001011010001100001000110100110001001100011001100010100010111000", 2)
n = 100

b = long_to_list(eps, n)

print(frequency_test(b, n))

True


In [4]:
# Block frequency test

def block_test(b: list, n: int, block_len: int) -> bool:

    blocks = n // block_len

    chunks = [ b[i:i+block_len] for i in range(0, n, block_len) ]

    s = [ sum(chunk) / blocks for chunk in chunks ]

    sobs = 4 * blocks * sum([ ((si - 0.5) ** 2) for si in s ])
    
    p = 1 - gammainc(blocks / 2, sobs / 2)
 
    return p >= 0.01

eps = int("1100100100001111110110101010001000100001011010001100001000110100110001001100011001100010100010111000", 2)
n = 100
m = 10

print(block_test(long_to_list(eps, n), n, m))

True


In [5]:
# runs test

def runs_test(b: list, n: int) -> bool:
    pi = sum(b) / n

    vobs = 1
    for i in range(1, n):
        if b[i] != b[i-1]:
            vobs += 1

    p = math.erfc(abs(vobs - 2 * n * pi * (1 - pi)) / (2 * (2 * n) ** 0.5 * pi * (1 - pi)))

    return p >= 0.01


eps = int("1100100100001111110110101010001000100001011010001100001000110100110001001100011001100010100010111000", 2)

print(runs_test(long_to_list(eps, 100), 100))

True


In [6]:
# Longest run of ones in a block test

def longest_run_test(b: list, n: int) -> bool:

    Ms = [8, 128, 10000]
    Ks = [3, 5, 6]
    Ns = [16, 49, 75]
    minL = [1, 4, 10]
    maxL = [4, 9, 16]
    PIs = [ [0.2148, 0.3672, 0.2305, 0.1875], [0.1174, 0.2430, 0.2493, 0.1752, 0.1027, 0.1124], [0.0882, 0.2092, 0.2483, 0.1933, 0.1208, 0.0675, 0.0727]]

    variant = None
    if n >= 128:
        variant = 0
    if n >= 6272:
        variant = 1
    if n >= 750000:
        variant = 2

    assert variant != None

    M = Ms[variant]
    K = Ks[variant]
    N = Ns[variant]
    lower = minL[variant]
    upper = maxL[variant]
    PI = PIs[variant]

    chunks = [ b[i:i+M] for i in range(0, n, M) ]
    counts = [0] * (K + 1)

    for chunk in chunks:
        runs = [ len(run) for run in "".join([str(bit) for bit in chunk]).split("0") if run != "" ]
        vobs = max(runs)
        if vobs <= lower:
            counts[0] += 1
        elif vobs >= upper:
            counts[K] += 1
        else:
            counts[vobs - lower] += 1

    chi = 0
    for i in range(K + 1):
        chi += ((counts[i] - N * PI[i]) ** 2) / (N * PI[i])

    p = 1 - gammainc(K / 2, chi / 2)

    return p >= 0.01

eps = int("11001100000101010110110001001100111000000000001001001101010100010001001111010110100000001101011111001100111001101101100010110010", 2)
n = 128

print(longest_run_test(long_to_list(eps, n), n))



True


In [7]:
# non-overlapping template matching test

def template_test(b: list, n: int, M: int, template: list) -> bool:
    N = n // M
    m = len(template)

    chunks = [ b[i:i+M] for i in range(0, n, M) ]
    count = [0] * N

    for i in range(N):
        start = 0
        end = start + m
        found = 0
        
        while end < len(chunks[i]):
            if chunks[i][start:end] == template:
                found += 1
                start = end
                end = start + m
            else:
                start += 1
                end += 1
        
        count[i] = found

    mu = (M - m + 1) / (2 ** m)
    sigma = M * (1 / (2 ** m) - (2 * m - 1) / (2 ** (2 * m)))

    chi = 0
    for i in range(N):
        chi += ((count[i] - mu) ** 2) / (sigma ** 2)
    
    p = 1 - gammainc(N / 2, chi / 2)

    return p >= 0.01

b = r.randbytes(2**7)
b = long_to_list(bytes_to_long(b), 2**10)
n = 2**10
pattern = [0, 0, 0, 0, 0, 0, 0, 0, 1]

print(template_test(b, n, 25, pattern))


True


In [9]:
# overlapping template matching test

def overlapping_template_test(b: list, n: int, M: int, template: list) -> bool:
    N = n // M
    m = len(template)

    chunks = [ b[i:i+M] for i in range(0, n, M) ]
    count = [0] * 6

    for chunk in chunks:
        start = 0
        end = start + m
        found = 0
        
        while end < len(chunk):
            if chunk[start:end] == template:
                found += 1
            start += 1
            end += 1
        
        if found <= 5:
            count[found] += 1
        else:
            count[5] += 1

    lambd = (M - m + 1) / (2 ** m)
    eta = lambd / 2

    pi = [0.364091, 0.185659, 0.139381, 0.100571, 0.0704323, 0.139865]

    chi = 0
    for i in range(6):
        chi += ((count[i] - N * pi[i]) ** 2) / (N * pi[i])

    p = 1 - gammainc(N / 2, chi / 2)

    return p >= 0.01

b = r.randbytes(2**7)
b = long_to_list(bytes_to_long(b), 2**10)
n = 2**10
pattern = [0, 0, 0, 0, 0, 0, 0, 0, 1]

print(overlapping_template_test(b, n, 25, pattern))


True


In [10]:
# serial test

def serial_test(b: list, n: int, m: int) -> bool:

    counts = {}

    chi = [0,0,0]

    for k in range(3):
        for i in range(2 ** (m - k)):
            start_bits = b[:m-k]
            updated_bits = b + start_bits

            tmpl = long_to_list(i, m - k)

            index = bin(i)[2:].zfill(m - k)
            if counts.get(index, None) != None:
                continue
            l = len(tmpl)
            if l == 0:
                continue
            for j in range(len(updated_bits) - l):
                if updated_bits[j:j+l] == tmpl:
                    counts[index] = counts.get(index, 0) + 1
        chi[k] = (2 ** (m - k)) / n * sum([ counts[t] ** 2 for t in counts if len(t) == m - k ]) - n

    gPhi = chi[0] - chi[1]
    hPhi = chi[0] - 2 * chi[1] + chi[2]

    pval1 = 1 - gammainc(2 ** (m - 2), gPhi / 2)
    pval2 = 1 - gammainc(2 ** (m - 3), hPhi / 2)


n = 10
eps = int("0011011101", 2)
m = 3

serial_test(long_to_list(eps, n), n, m)

In [11]:
# cumsum

from math import erf, floor

def phi(x):
    return (1.0 + erf(x / (2 ** 0.5))) / 2.0

def cumsum_test(b: list, n: int, forward: bool) -> bool:
    if forward:
        b = [x for x in reversed(b)]

    S = [0] * (n + 1)
    for i in range(n):
        S[i + 1] = S[i] + b[i] * 2 - 1

    Z = max([abs(s) for s in S])

    print(Z)

    p1 = 0
    p2 = 0

    start1 = floor(((-n / Z) + 1) / 4)
    end1 = floor(((n / Z) - 1) / 4)

    start2 = floor(((-n / Z) - 3) / 4)
    end2 = floor(((n / Z) - 1) / 4)

    for k in range(start1, end1):
        p1 += phi((4 * k + 1) * Z / n ** 0.5) - phi((4 * k - 1) * Z / n ** 0.5)

    for k in range(start2, end2):
        p2 += phi((4 * k + 3) * Z / n ** 0.5) - phi((4 * k + 1) * Z / n ** 0.5)

    p = 1 - p1 + p2

    return p >= 0.01


n = 100
eps = int("1100100100001111110110101010001000100001011010001100001000110100110001001100011001100010100010111000", 2)

cumsum_test(long_to_list(eps, n), n, True)

16


True

In [27]:
import clr

RuntimeError: Failed to create a default .NET runtime, which would
                    have been "mono" on this system. Either install a
                    compatible runtime or configure it explicitly via
                    `set_runtime` or the `PYTHONNET_*` environment variables
                    (see set_runtime_from_env).